# **📋 Pneumonia Detection with ResNet & PyTorch**

### **Project Goal**
To build a Deep Learning model that looks at Chest X-Ray images and predicts if a patient has **Pneumonia** or is **Normal**.

### **1. Imports and Setup**
*   **`torch` & `torchvision`**: The main PyTorch libraries for building AI and handling images.
*   **`os`**: To navigate files on your computer.
*   **`matplotlib` & `seaborn`**: For drawing graphs and showing images.

In [18]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm

Makes the chart appear directly inside the notebook

In [19]:
# %matplotlib inline

### **2. Configuration (Constants)**
*   **`IMAGE_SIZE = 150`**: Neural Networks need fixed inputs. We will squash all images (big or small) to 150x150 pixels.
*   **`BATCH_SIZE = 32`**: The "Mini-Batch" size. The model will study 32 images at a time before updating its brain.
*   **`DATA_DIR`**: The path where your folders (train/test/val) live.

In [20]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/chest_xray')

KeyboardInterrupt: 

In [ ]:
# CONSTANTS
IMAGE_SIZE = 150
BATCH_SIZE = 16 # 32 for later
DATA_DIR = '/content/drive/MyDrive/chest_xray'

In [ ]:
DATA_DIR

### **3. Data Transforms (The Preprocessing Pipeline)**
Before an image enters the model, it must be "cleaned" and prepared. 

*   **`Grayscale(3)`**: 
     *   *Concept:* X-rays are Black & White (1 channel), but ResNet expects Color (3 channels).
     *   *Trick:* We duplicate the B&W layer 3 times to "fake" a color image so ResNet accepts it.

*   **`Resize`**: Ensures every image is exactly 150x150.

*   **`ToTensor`**: Converts the image from **Pixels (0-255)** to **Math Numbers (0.0-1.0)**.

*   **`Normalize`**: Shifts the math numbers to be between **-1 and 1**. This helps the model learn faster (stable gradients).

In [ ]:
# TRANSFORMERS (grayscale, resize, to tensor, normalize)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

### **4. Loading Data (The "Bridge")**
*   **`ImageFolder`**: PyTorch's magic tool. It looks at folder names ("Normal", "Pneumonia") and automatically converts them into labels (0, 1).
*   **`DataLoader`**: The "Delivery Truck". It gathers images from the hard drive, packages them into batches of 32, and delivers them to the GPU.
    *   **`shuffle=True` (Train)**: Critical! We mix up the cards so the model doesn't memorize the order.     
    *   **`shuffle=False` (Test)**: We don't need to shuffle when testing.

In [ ]:
# LOAD DATASETS

train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("✅ Classes : ", train_dataset.classes)
print("✅ Dataset sizes : Train", len(train_dataset))
print("✅ Dataset sizes : Validation", len(val_dataset))
print("✅ Dataset sizes : Test", len(test_dataset))

In [ ]:
print(len(train_dataset))

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### **5. Data Visualization (Checking for Imbalance)**
*   **The Problem:** In medical data, "Disease" cases are often rare. If we have 1000 Normal and only 50 Pneumonia, the model might just guess "Normal" every time.
*   **The Fix:** We plot a bar chart to see if we need to fix this (e.g., using Weighted Loss later).
 
**🚀 PERFORMANCE TIP:** 
*   I replaced your old code `[label for...]` which loaded *every image pixel* (taking minutes).
*   I used `train_dataset.targets` which reads *only the labels* (taking milliseconds).

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

# FAST WAY: Access the labels directly from the list stored in memory
# This takes 0.01 seconds instead of 10 minutes
labels = train_dataset.targets 
label_counts = Counter(labels)

class_names = train_dataset.classes
class_labels = [class_names[i] for i in label_counts.keys()]
counts = list(label_counts.values())

# PLOTTING
plt.figure(figsize=(6, 4))
plt.bar(class_labels, counts, color=['green', 'red'])
plt.title("Class Distribution in Training Set")
plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.show()

### **6. Sanity Check (View the Images)**
*   **Concept:** Always look at your data! Did the resize work? Is "Pneumonia" actually labeled correctly?
*   **`unnormalize`**: The images are currently number ranges [-1, 1] (weird colors). We calculate `img * 0.5 + 0.5` to bring them back to  so our eyes can see them normally.

In [ ]:
def show_batch(dl, class_names):
    
    # Fetch one batch from the dataloader
    images, labels = next(iter(dl))
    fig, axes = plt.subplots(3, 8, figsize=(15, 6))

    # Show images up to batch size
    num_images = len(images)
    for i, ax in enumerate(axes.flatten()):
        if i < num_images:
            img = images[i].numpy().transpose((1, 2, 0))
            
            img = (img * 0.5) + 0.5  # unnormalize 
            ax.imshow(img)
            ax.set_title(class_names[labels[i]])
            ax.axis('off')
        else:
            ax.axis('off')
    plt.suptitle("Sample Images from Training Set", fontsize=16)
    plt.tight_layout()
    plt.show()

show_batch(train_loader, train_dataset.classes)

In [ ]:
from torchvision import models

model = models.resnet18(weights="IMAGENET1K_V1")
model.conv_block1 = nn.Conv2d(1, 64, kernel_size = 7, stride=2, padding=3, bias=False)

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 18 * 18, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

In [ ]:
import torch.optim as optim

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# initialize model 
model = PneumoniaCNN().to(device)

# handle class imbalance : calculate weights
labels = [label for _, label in train_dataset.imgs]
class_counts = Counter(labels)
total = sum(class_counts.values())
weights = [total / class_counts[i] for i in range(len(class_counts))]

# Define loss function with weights
class_weights = torch.FloatTensor(weights).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

In [ ]:


def train_model(model, loader):
    model.train()
    total_loss, correct = 0, 0


    # add progress bar

    pbar = tqdm(loader, desc='Training', unit='batch')

    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() 
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()

        # update progress bar with current loss and accuracy
        pbar.set_postfix({'loss': total_loss / (len(loader)), 'accuracy': correct / ((len(loader.dataset)))})

    accuracy = correct / len(loader.dataset)
    return total_loss / len(loader), accuracy

In [ ]:
def validate_model(model, loader):
    model.eval()
    total_loss, correct = 0, 0

    # ← Add progress bar
    pbar = tqdm(loader, desc="Validating", leave=False)

    with torch.no_grad():
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item() 
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()

            pbar.set_postfix({'loss': total_loss / (len(loader)), 'accuracy': correct / ((len(loader.dataset)))})

    accuracy = correct / len(loader.dataset)
    return total_loss / len(loader), accuracy

In [ ]:
NUM_EPOCHS = 5 ## change to 25 for better results
best_val_accuracy = 0.0

train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []


for epoch in range(NUM_EPOCHS + 1):
    train_loss, train_accuracy = train_model(model, train_loader)
    val_loss, val_accuracy = validate_model(model, val_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    # adjust learning rate
    scheduler.step(val_accuracy)

    print(f"Epoch {epoch}/{NUM_EPOCHS}  "
          f"Train Loss: {train_loss:.4f}, Acc: {train_accuracy:.4f}  "
          f"Val Loss: {val_loss:.4f}, Acc: {val_accuracy:.4f}", flush=True)
    
    # save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_pneumonia_cnn.pth')
        print("=> Saved Best Model", flush=True)

print(f"🎯 Best validation Accuracy: {best_val_accuracy:.4f}")

In [ ]:
epochs = range(1, NUM_EPOCHS + 1)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Train Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.title('Loss over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Train Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.show()

In [ ]:
IMAGE_SIZE = 150

# safer medical augmentation for training

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),

    transforms.RandomRotation(degrees=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [ ]:
test_val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [ ]:
# paths to data
data_dir = 'data/chest_xray'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=test_val_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=test_val_transform)

BATCH_SIZE = 16 # can increase to 32 later
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

class_names = train_dataset.classes
print("✅ Classes : ", class_names)

In [ ]:
# get a batch of training data

images, labels = next(iter(train_loader))

# denormalize images for visualization
def denormalize(img):
    img = img * 0.5 + 0.5  # unnormalize
    img = img.numpy().transpose((1, 2, 0))
    return img


# plot a grid of images
plt.figure(figsize=(12, 6))
for i in range(min(BATCH_SIZE, 16)):
    plt.subplot(4, 4, i + 1)
    plt.imshow(denormalize(images[i]))
    plt.title(class_names[labels[i]])
    plt.axis('off')

plt.tight_layout()
plt.show()